In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta, date, time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from config.config_loader import Config

config = Config()

In [3]:
estado = "C2"
perd_path = config.get_path("data_path.elec.processed.perd_total")

In [4]:
# Parameters
estado = "C2"
perd_path = "c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\processed\\elec\\perdidas.pkl"


In [5]:
from shared.IgnisAPI import IgnisAPI

api_comer = IgnisAPI(token='BWBCPQIEHO', version=0)

In [6]:
columns = ['20_td', '30_td', '61_td', '62_td', '63_td', '64_td']

In [7]:
df_merged = pd.DataFrame()

if os.path.exists(perd_path):
    df_perd = pd.read_pickle(perd_path)
    start_date = ((df_perd['datetime'].max()) + timedelta(days=1))
    start_date = start_date.date()
else:
    df_perd = pd.DataFrame()
    start_date = date(2023,1,1)
    
start_date = datetime.combine(start_date, time(0,0,0))
end_date = datetime.now()
end_date =  end_date.date()
end_date = datetime.combine(end_date, time(23,0,0))
date_range = (end_date - start_date + timedelta(days=1)).days
print(date_range)
print(start_date)
print(end_date)

if start_date <= end_date:
    for i in range (6):
        perd = api_comer.fetch_full_data_range(start=start_date.date(),
                                        end=end_date.date(), 
                                        endpoint='/mirlo/api/liquidaciones-comer-mercados-cierres/',
                                        comer=True,
                                        estado=estado,
                                        id = -(i+1)
                                            )
        if perd.empty:
            break
        perd['VALOR'] = perd['VALOR'] / 100
        perd = perd.rename(columns={'VALOR': columns[i]})
        if i == 0:
            df_merged = perd
        else:
            df_merged = pd.merge(df_merged, perd, 
                                    on=['FECHA', 'PERIODO', 'ESTADOLIQ', 'QH', 'ID', 'GEO_ID'], 
                                    how='outer')
        
    if not df_merged.empty:

        ds = df_merged.groupby(['FECHA', 'PERIODO']).sum()
        df = pd.DataFrame(ds)
        df = df.reset_index()
        df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
        df = df.drop(['ESTADOLIQ', 'QH', 'ID', 'GEO_ID'], axis=1)

        date_counts = df.groupby('FECHA').size()

        dates_with_23_rows = date_counts[date_counts == 23].index
        condition_date_23_rows = df['FECHA'].isin(dates_with_23_rows)
        dates_with_25_rows = date_counts[date_counts == 25].index
        condition_date_25_rows = df['FECHA'].isin(dates_with_25_rows)
        dates_with_24_rows = date_counts[date_counts == 24].index
        condition_date_24_rows = df['FECHA'].isin(dates_with_24_rows)

        condition_hour_gte_1 = df['PERIODO'] < 3
        condition_hour_gte_2 = df['PERIODO'] >= 3

        df.loc[condition_date_23_rows & condition_hour_gte_1, 'datetime'] = df.loc[condition_date_23_rows & condition_hour_gte_1].apply(
            lambda row: datetime(row['FECHA'].year, row['FECHA'].month, row['FECHA'].day, row['PERIODO'] - 1),
            axis=1)

        df.loc[condition_date_23_rows & condition_hour_gte_2, 'datetime'] = df.loc[condition_date_23_rows & condition_hour_gte_2].apply(
            lambda row: datetime(row['FECHA'].year, row['FECHA'].month, row['FECHA'].day, row['PERIODO']),
            axis=1)

        condition_hour_gte_1 = df['PERIODO'] <= 3
        condition_hour_gte_2 = df['PERIODO'] > 3

        df.loc[condition_date_25_rows & condition_hour_gte_1, 'datetime'] = df.loc[condition_date_25_rows & condition_hour_gte_1].apply(
            lambda row: datetime(row['FECHA'].year, row['FECHA'].month, row['FECHA'].day, row['PERIODO'] - 1),
            axis=1)

        df.loc[condition_date_25_rows & condition_hour_gte_2, 'datetime'] = df.loc[condition_date_25_rows & condition_hour_gte_2].apply(
            lambda row: datetime(row['FECHA'].year, row['FECHA'].month, row['FECHA'].day, row['PERIODO'] - 2),
            axis=1)

        df.loc[condition_date_24_rows, 'datetime'] = df.loc[condition_date_24_rows].apply(
            lambda row: datetime(row['FECHA'].year, row['FECHA'].month, row['FECHA'].day, row['PERIODO'] - 1),
            axis=1)

        df_perd_new = df[['datetime', '20_td', '30_td', '61_td', '62_td', '63_td', '64_td']]
        df_perd_new['datetime'] = pd.to_datetime(df_perd_new['datetime'], errors='coerce')

        df_perd = pd.concat([df_perd, df_perd_new], ignore_index=True)
    else:
        print("I'm empty")

55
2025-05-01 00:00:00
2025-06-24 23:00:00
hi


hi


hi


hi


hi


hi


C:\Users\mikel.perez\AppData\Local\Temp\ipykernel_32412\4021612206.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perd_new['datetime'] = pd.to_datetime(df_perd_new['datetime'], errors='coerce')


In [8]:
df_perd.to_pickle(perd_path)